In [2]:
from numpy import product
import pandas as pd



#Importando data de análisis desde GitHub:

paises = pd.read_csv("https://raw.githubusercontent.com/BryanHndz/ProyectoBI/main/Data/PaisesProyecto.csv")
sucursales = pd.read_csv("https://raw.githubusercontent.com/BryanHndz/ProyectoBI/main/Data/SucursalesProyecto.csv")
vendedores = pd.read_csv("https://raw.githubusercontent.com/BryanHndz/ProyectoBI/main/Data/VendedoresProyecto.csv")
compras = pd.read_csv("https://raw.githubusercontent.com/BryanHndz/ProyectoBI/main/Data/ComprasProyecto.csv")
ventas = pd.read_csv("https://raw.githubusercontent.com/BryanHndz/ProyectoBI/main/Data/VentasProyecto.csv")
productos = pd.read_csv("https://raw.githubusercontent.com/BryanHndz/ProyectoBI/main/Data/Producto.csv")


In [3]:
# PRIMER ANÁLISIS: Top 5 vendedores con más ventas:

# 1) Primero debemos agrupar y sumar las ventas que tengan el mismo id de vendedor
SumadeVentasPorIdVendedor = ventas.groupby('idVendedor').size().reset_index(name='NumeroDeVentas')

# 2) Luego Debemos hacer un top 5 sobre esas sumas
Top5VentasPorId = SumadeVentasPorIdVendedor.nlargest(5,'NumeroDeVentas')

# 3) Luego asociar el nombre de cada vendedor con respecto al id que tienen.
Top5Vendedores = pd.merge(Top5VentasPorId,vendedores,left_on='idVendedor',right_on = 'id')

# 4) Imprimir el Top 5 de vendedores con más ventas de la compañía:
Top5Vendedores[['idVendedor','first_name','last_name','NumeroDeVentas','fecha_salida']]

# Conclusión: El resultado muestra que los mejores vendedores siguen con nosotros y que únicamente uno de ellos ha abandonado la empresa
# Sería buena propuesta incentivar a esos vendedores estrella.

,idVendedor,first_name,last_name,NumeroDeVentas,fecha_salida
0,83,Pen,Medmore,67,NaN
1,47,Eamon,Brandom,64,NaN
2,86,Urbain,Sibille,64,NaN
3,94,Von,O'Lennachain,62,03/15/2021
4,9,Cristi,Myott,60,NaN


In [4]:
from IPython.utils.text import string
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
#SEGUNDO ANÁLISIS: Sucursal con mejores ventas.

# 0) Asociar cada venta realizada con el precio del producto al que hace referencia:
VentasXProducto = pd.merge(ventas, productos, left_on='idProducto', right_on='id')

VentasXProducto[['idProducto','idSucursal','nombre','precio']]

# 1) Agrupar y sumar todas las ventas que hay en los registros mediante el id de sucursal que poseen y sumar el valor total de esas ventas.

VentasXProducto['precio'] = VentasXProducto['precio'].str.replace('$', '').astype(float) #Convirtiendo primero dato "precio" a númerico

SucursalSuma = VentasXProducto.groupby('idSucursal')['precio'].sum().reset_index()

# 2) Hacer un top 4 (porque hay solo cuatro sucursales) de las mejores tiendas:
TopSucursales = SucursalSuma.nlargest(4,'precio')

# 3) Haciendo la correspondencia entre el id de la sucursal y el nombre, para la mejor comprensión de los datos:
TopSucursalesFinal = pd.merge(TopSucursales, sucursales, left_on='idSucursal', right_on='id')

# 3.1) Convirtiendo columna "precio" a un formato más compresible de dinero:
TopSucursalesFinal['precio'] = TopSucursalesFinal['precio'].map(lambda x: locale.currency(x, grouping=True))

TopSucursalesFinal[['idSucursal','nombre','precio']]
#Se concluye que la mejor tienda de Vehículos es la "DIDEA SV" pero todas las tiendas tienen un buen desempeño.

<ipython-input-4-44774f3fc3b9>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  VentasXProducto['precio'] = VentasXProducto['precio'].str.replace('$', '').astype(float) #Convirtiendo primero dato "precio" a númerico


,idSucursal,nombre,precio
0,1,DIDEA SV,"$50,155,792.98"
1,2,Excel Automotriz GT,"$49,902,654.79"
2,4,Global Autos HN,"$48,870,366.47"
3,3,Veinsa Motors CR,"$46,614,000.50"


In [17]:
#TERCER ANALISIS: Vehículo más vendido.

# 1) Agrupar la tabla "ventas" por idProducto y contar las repeticiones
TopVehiculosMasVendidos = ventas.groupby('idProducto').size().reset_index(name='NumeroDeVentas')

# 2) Encontrar el top 1 de vehículo más vendido:
IdVehiculoMasVendido = TopVehiculosMasVendidos.nlargest(1,'NumeroDeVentas')

# 3) Asociar id del vehículo más vendido con la tabla de productos para encontrar su nombre
VehiculoMasVendido = pd.merge(IdVehiculoMasVendido,productos,left_on='idProducto',right_on='id')

# 4) Imprimir información de vehículo más vendido:
VehiculoMasVendido[['id','nombre','precio','NumeroDeVentas']]

,id,nombre,precio,NumeroDeVentas
0,4,Sable,$18097.80,548


In [56]:
#CUARTO ANÁLISIS: Ganancias brutas obtenidas en el año 2019; esto será calculado a través de la diferencia entre las ventas de
# ese año menos las compras en el mismo periodo del 1 de enero de 2019 al 31 de diciembre 2019.
from datetime import datetime

# 1) Sumar las compras realizadas con fecha en 2019 en un nuevo DataFrame:
compras['fecha'] = pd.to_datetime(compras['fecha']) #compras['precio'] = pd.to_numeric(compras['precio'].str.replace('$', ''))
Compras2019 = compras.groupby(compras['fecha'].dt.year)['precio'].sum().reset_index(name='total compras')
#Merge['total compras'] = Merge['total compras'].map(lambda x: locale.currency(x, grouping=True))

Compras2019
# 2) Sumar las ventas realizadas con fecha en 2019 en un nuevo DataFrame:
VentasXProd = pd.merge(productos,ventas,left_on='id',right_on='idProducto')
VentasXProd['precio'] = VentasXProd['precio'].str.replace('$', '').astype(float)

#Aplicando la resta del descuento en cada vehículo:
VentasXProd['PrecioConDesc'] = VentasXProd['precio'] - (VentasXProd['precio'] * VentasXProd['Descuento'])
VentasXProd['Fecha'] = pd.to_datetime(VentasXProd['Fecha'])

Ventas2019 = VentasXProd.groupby(VentasXProd['Fecha'].dt.year)['PrecioConDesc'].sum().reset_index(name='total ventas')

Merge = pd.merge(Ventas2019,Compras2019,left_on='Fecha',right_on='fecha')

Merge['Ganancias'] = Merge['total ventas'] - Merge['total compras']
Merge['total compras'] = Merge['total compras'].map(lambda x: locale.currency(x, grouping=True))
Merge['total ventas'] =  Merge['total ventas'].map(lambda x: locale.currency(x, grouping=True))
Merge['Ganancias'] =  Merge['Ganancias'].map(lambda x: locale.currency(x, grouping=True))

Ganancias2019 = Merge[ Merge['Fecha'] == 2019 ]

Ganancias2019[['Fecha','total ventas','total compras','Ganancias']]
#Lo cual indica que hubieron ganancias positivas (de $29,019,290.77) para la corporación durante el año 2019

<ipython-input-56-80889afd0368>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  VentasXProd['precio'] = VentasXProd['precio'].str.replace('$', '').astype(float)


,Fecha,total ventas,total compras,Ganancias
1,2019,"$33,963,030.06","$4,943,739.29","$29,019,290.77"


In [84]:
#ULTIMO ANÁLISIS: CALCULAR LA MASA SALARIAL DE LOS VENDEDORES DE LA CORPORACIÓN ENTERA:

# 1) Sumar el total de salario de los vendedores y redondearlo a dos decimales:
MasaSalarial = vendedores['salario'].sum().round(2)

# 2) Darle formato de divisa:
MasaSalarialFinal = '${:,.2f}'.format(MasaSalarial)

# 3) Imprimiento el resultado:
print("La masa salarial total de vendedores es de: " + MasaSalarialFinal)

La masa salarial total de vendedores es de: $465,475.73
